# Deploy Your Own Model in HuggingFace

## Introduction
The session will guide participants through training a machine learning model for pneumonia classification using chest radiographs and then demonstrate how to create an interactive demo for this model using Gradio.


## Requirements

Participants should be confortable in reading Python code.



We will begin by installing a few necessary packages. Python packages are easily installed using `pip` from the command line.

## Installation of Python Packages
In this section, we install several Python packages that are essential for our project:
- `timm`: For accessing pre-trained models and image processing tools.
- `cv2`: Open CV is an image processing library.
- `pytorch`: deep learning framework with functions needed to train models.
- `gradio`: A key package for creating interactive web interfaces for machine learning models.
- `pandas`: most used library to deal with tables (Data Frames).

These packages are installed using Python's package manager, `pip`. It's a straightforward way to add new functionality to our Python environment.



In [1]:
!pip install -q timm==0.9.10 kaleido cohere openai tiktoken gradio==4.7.1 typing_extensions

Next, let's download a small dataset of chest radiographs, sampled from the RSNA Pneumonia Detection Challenge dataset.

These images have been converted from DICOM to PNG format for ease of use.

In [2]:
!rm -rf tiny-pneumonia*
!wget -q -O tiny-pneumonia.zip https://www.dropbox.com/scl/fi/egb1pidyn8rnt79qt6lno/tiny-pneumonia.zip?rlkey=xzaci43atj6mqqpqofp3ye0if&dl=0
!unzip -q tiny-pneumonia.zip
!ls

pneumonia_model.pth  sample_data  tiny-pneumonia  tiny-pneumonia.zip


Now, let's write some Python code. We start by importing some standard libraries commonly used in deep learning pipelines.

Albumentations is a popular library for data augmentation, which is plays an important role in training deep learning models.

We will use PyTorch to build and train our models. In addition, we will use the popular `timm` library, which contains an extensive selection of pretrained deep learning models.

In [3]:
import albumentations as A
import cv2
import glob
import numpy as np
import pandas as pd
import torch
import timm

from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

## Dataset Preparation
Here, we focus on preparing the dataset for our pneumonia classification model. The dataset consists of chest radiographs and is a sample from the RSNA Pneumonia Detection Challenge. For practicality, the images have been converted from the medical imaging format DICOM to the more universally used PNG format. This simplification allows easier manipulation and processing of the images in Python.



Let's take a look at our dataset.

In [4]:
images = glob.glob("tiny-pneumonia/images/*")
df = pd.read_csv("tiny-pneumonia/labels.df")
df["filename"] = "tiny-pneumonia/images/" + df.filename
print(f"There are {len(df)} images.")
print(f"The prevalence of pneumonia is {df.opacity.mean()*100:0.1f}%.")
df.head()

There are 1000 images.
The prevalence of pneumonia is 50.0%.


,patientId,opacity,filename
0,00a85be6-6eb0-421d-8acf-ff2dc0007e8a,0,tiny-pneumonia/images/00a85be6-6eb0-421d-8acf-...
1,01027bc3-dc40-4165-a6c3-d6be2cb7ca34,0,tiny-pneumonia/images/01027bc3-dc40-4165-a6c3-...
2,01aad2a6-3b93-45e3-bf37-2d73348cb6fc,0,tiny-pneumonia/images/01aad2a6-3b93-45e3-bf37-...
3,056fb495-574d-4b37-9f89-77b9ced2ef43,0,tiny-pneumonia/images/056fb495-574d-4b37-9f89-...
4,07290e87-0c95-4daf-ae97-2f7f971f23c0,0,tiny-pneumonia/images/07290e87-0c95-4daf-ae97-...


Let's split our dataset into 3 partitions: 1) training, 2) validation, and 3) test.

A common distribution is 70% training, 10%% validation, and 20% test.

The training set is used to train the model by optimizing its weights (parameters).

The validation set is used to tune various parameters during model training, such as the learning rate, batch size, model architecture, and much more.

The test set is used to evaluate the performance of the model and should only be used for this purpose to avoid overfitting. Typically, many experiments are conducted on the training and validation sets to create the best model before finally using the test set to evaluate the performance.

A familiar analogy may help your understanding. If you imagine studying for boards, the training set is a bank of practice questions, the validation set is a practice test, and the test set is the real exam.

In [5]:
all_indices = np.arange(len(df))
test_indices = np.random.choice(all_indices, int(0.2 * len(all_indices)),
                                replace=False)
val_indices = np.random.choice(list(set(all_indices) - set(test_indices)),
                               int(0.1 * len(all_indices)), replace=False)
train_indices = list(set(all_indices) - set(test_indices) - set(val_indices))

df.loc[train_indices, "split"] = "train"
df.loc[val_indices, "split"] = "val"
df.loc[test_indices, "split"] = "test"

train_df = df[df["split"] == "train"].reset_index(drop=True)
val_df = df[df["split"] == "val"].reset_index(drop=True)
test_df = df[df["split"] == "test"].reset_index(drop=True)

df["split"].value_counts()

train    700
test     200
val      100
Name: split, dtype: int64

Here are a basic set of hyperparameters that you can play around with.

In [6]:
MODEL_ARCHITECTURE = "tf_efficientnetv2_s"
IMAGENET_PRETRAINED = True
IMAGE_SIZE = (256, 256) # height, width
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_EPOCHS = 10
DATA_AUGMENTATION = True

Here we define the data augmentation policy for training, as well as the transforms that will be used during inference. In general, the same resizing parameters (e.g., image width and height) and normalization method should be applied during both training and validation.

In [7]:
val_transforms = A.Compose([
  A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1], p=1),
  A.Normalize(mean=0.5, std=0.5, p=1)
], p=1)

if DATA_AUGMENTATION:
  train_transforms = A.Compose([
    A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1], p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.Normalize(mean=0.5, std=0.5, p=1)
  ], p=1)
else:
  train_transforms = val_transforms

Here we define a simple PyTorch dataset. It works by retrieving 1 image and applying the transforms we defined above.

In [8]:
class ImageDataset(Dataset):

  def __init__(self, inputs, labels, transforms):
    self.inputs = inputs
    self.labels = labels
    self.transforms = transforms

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, i):
    img = cv2.imread(self.inputs[i], 0)
    img = self.transforms(image=img)["image"]
    img = np.expand_dims(img, axis=0)
    return img, np.expand_dims(self.labels[i], axis=-1)

Here we load a pretrained model from `timm` and create our dataset classes for each of the partitions (training, validation, and test). We also define the optimizer and learning rate scheduler, which are essential to training.

In [9]:
model = timm.create_model(MODEL_ARCHITECTURE, num_classes=1,
                          pretrained=IMAGENET_PRETRAINED, in_chans=1)
model = model.train().cuda()

train_ds = ImageDataset(train_df.filename.values, train_df.opacity.values,
                        train_transforms)
val_ds = ImageDataset(val_df.filename.values, val_df.opacity.values,
                      val_transforms)
test_ds = ImageDataset(test_df.filename.values, test_df.opacity.values,
                       val_transforms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=2)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True,
                         num_workers=2)

optimizer = torch.optim.AdamW(lr=LEARNING_RATE, params=model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
              T_max=len(train_ds) * NUM_EPOCHS // BATCH_SIZE, eta_min=0)
loss_fn = torch.nn.BCEWithLogitsLoss()

Here we define our training loop. We evaluate our models on the validation set after each training epoch, monitoring the AUC value.

In [10]:
for each_epoch in range(NUM_EPOCHS):
  model.train()
  pbar = tqdm(train_loader)
  for batch in pbar:
    X, y = batch
    X, y = X.cuda(), y.cuda()
    optimizer.zero_grad()
    p = model(X)
    loss = loss_fn(p.float(), y.float())
    loss.backward()
    optimizer.step()
    scheduler.step()
    pbar.set_postfix({"loss": loss.item()})
  val_preds, val_gt, val_losses = [], [], []
  model.eval()
  for batch in val_loader:
    X, y = batch
    X, y = X.cuda(), y.cuda()
    with torch.no_grad():
      p = model(X)
    val_preds.append(torch.sigmoid(p).cpu().numpy())
    val_gt.append(y.cpu().numpy())
    val_losses.append(loss_fn(p.float(), y.float()).item())
  val_preds = np.concatenate(val_preds)
  val_gt = np.concatenate(val_gt)
  val_auc = roc_auc_score(val_gt, val_preds)
  print(f"EPOCH {each_epoch + 1} VAL AUC : {val_auc:0.3f}")

  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH 1 VAL AUC : 0.879


  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH 2 VAL AUC : 0.900


  0%|          | 0/22 [00:00<?, ?it/s]

...

EPOCH 8 VAL AUC : 0.933


  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH 9 VAL AUC : 0.934


  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH 10 VAL AUC : 0.936


Once we have our trained model, we evaluate it on the test set. Usually, we will perform many additional experiments during training and validation before deciding on the best model that we will finally test. In this case, our basic model is already doing quite well.

In [11]:
test_preds, test_gt, test_losses = [], [], []
model.eval()
for batch in test_loader:
  X, y = batch
  X, y = X.cuda(), y.cuda()
  with torch.no_grad():
    p = model(X)
  test_preds.append(torch.sigmoid(p).cpu().numpy())
  test_gt.append(y.cpu().numpy())
  test_losses.append(loss_fn(p.float(), y.float()).item())
test_preds = np.concatenate(test_preds)
test_gt = np.concatenate(test_gt)
test_auc = roc_auc_score(test_gt, test_preds)
print(f"TEST AUC : {test_auc:0.3f}")

TEST AUC : 0.956


Let's save the model weights.

In [12]:
torch.save(model.state_dict(), "pneumonia_model.pth")

Now, we will deploy our model so people can test it. We will load the saved model weights into the model that we created at the beginning. In addition, we will use the inference transformations to ensure that the input is in the expected format. Finally, we use some basic `gradio` classes to create a demo that allows users to upload images and interact with your model.

If you are interested in deploying this model publicly for free, you can use Hugging Face Spaces and simply copy and paste this code. You will also need to make sure the necessary packages are installed using a basic `requirements.txt` file.

In [ ]:
import albumentations as A
import gradio as gr
import timm
import torch


MODEL_ARCHITECTURE = "tf_efficientnetv2_s"
IMAGE_SIZE = (256, 256)

model = timm.create_model(MODEL_ARCHITECTURE, num_classes=1,
                          pretrained=False, in_chans=1)
model.load_state_dict(torch.load("pneumonia_model.pth"))
model = model.eval()

val_transforms = A.Compose([
  A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1], p=1),
  A.Normalize(mean=0.5, std=0.5, p=1)
], p=1)

def predict_pneumonia(Image):
  img = val_transforms(image=Image)["image"]
  img = torch.from_numpy(img).unsqueeze(0).unsqueeze(0)
  with torch.inference_mode():
    p = model(img)
  p = torch.sigmoid(p)[0][0].item()
  return f"Probability of Pneumonia: {p*100:0.1f}%"

image = gr.Image(image_mode="L")
label = gr.Label(show_label=True, label="Result")

demo = gr.Interface(
  fn=predict_pneumonia,
  inputs=[image],
  outputs=label
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a0a08db7237fe89d1d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
